202010370311391 - Ungga Putra Mahendra


In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
pip install requests

In [ ]:
pip install tqdm

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


Executing: /tmp/apt-key-gpghome.cjhyq6s9Ho/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.3JCWtSTiN1/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.p6ESw0mtFn/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:11 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [738 kB]
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from tqdm import tqdm
import os
import sys
from pathlib import Path
import requests
import threading

In [ ]:
# Setup Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

# Buat instance WebDriver
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
main_url = 'https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-madiun/kategori/narkotika-dan-psikotropika-1/tahunjenis/upload/tahun/2022'
# Variable pages digunakan untuk menyimpan semua page yang ditemukan pada URL utama
pages = set()

def main_func(url):
  temp = set()
  driver.get(url)
  all_links = driver.find_elements(By.CLASS_NAME, 'entry-c')

  # Loop melalui setiap elemen yang ditemukan
  for link in all_links:
    strong_elements = link.find_elements(By.TAG_NAME, 'strong')

    for strong_element in strong_elements:
      a_elements = strong_element.find_elements(By.TAG_NAME, 'a')

      for a_element in a_elements:
        page_url = a_element.get_attribute('href')

        if page_url:
          temp.add(page_url)

  return temp

pages = pages | main_func(main_url)

In [ ]:
for i in range(2, 5):
  temp_url = f'{main_url}/page/{i}.html'
  pages = pages | main_func(temp_url)

# Cetak semua URL yang telah ditemukan
for page in pages:
  print(page)

https://putusan3.mahkamahagung.go.id/direktori/putusan/zaed42daa82ee1cca6af313231373231.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaed28f77b97278eb04c313333383131.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaecf5eb30bbd64a945a313433313432.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaed74717a47f2449d0b313435303235.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaed859bbd739ed484ac313130353436.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaed6bcbde996ccc906f313434343437.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaed1d034a759ece9a56303833323238.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaed49f6a20bbfd69578313332353134.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaeca98577de040698d7303931323034.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaeca4e9497dd46aad27313232343030.html
https://putusan3.mahkamahagung.go.id/direktori/putusan/zaec79c656bfa7e

In [ ]:
def getpdflink(list_urls):
  hrefs = []
  # Loop melalui setiap link yang ada pada 'listed_pages'
  for link in list_urls:
    driver.get(link)

    # Mencari elemen <div> dengan ID 'collapseThree'
    collapseThree_element = driver.find_element(By.ID, 'collapseThree')

    # Mencari elemen <ul> dalam elemen <div>
    ul_element = collapseThree_element.find_element(By.TAG_NAME, 'ul')

    # Mencari semua elemen <li> dalam elemen <ul>
    li_elements = ul_element.find_elements(By.TAG_NAME, 'li')

    for li_element in li_elements:
        a_elements = li_element.find_elements(By.TAG_NAME, 'a')

        for a_element in a_elements:
            text = a_element.text
            # Menambahkan kondisi untuk memeriksa apakah teks diakhiri dengan ".pdf"
            if text.endswith('.pdf'):
                href = a_element.get_attribute('href')
                hrefs.append(href)

  return hrefs

In [ ]:
# Menampilkan semua link download pdf yang didapat
all_link = getpdflink(list(pages))
for href in all_link:
  print(href)

https://putusan3.mahkamahagung.go.id/direktori/download_file/72cead058a3208f584ce408d1508172c/pdf/zaed42daa82ee1cca6af313231373231
https://putusan3.mahkamahagung.go.id/direktori/download_file/0d8b90b5f29a3cb4092d781a41999be8/pdf/zaed28f77b97278eb04c313333383131
https://putusan3.mahkamahagung.go.id/direktori/download_file/6f33ce34df52ccb3ec9bdff546b9ba34/pdf/zaecf5eb30bbd64a945a313433313432
https://putusan3.mahkamahagung.go.id/direktori/download_file/868720ce2edeb222d4ba3134f1da0f01/pdf/zaed74717a47f2449d0b313435303235
https://putusan3.mahkamahagung.go.id/direktori/download_file/868c4c4d92f1dd8f237db3724c55bb06/pdf/zaed859bbd739ed484ac313130353436
https://putusan3.mahkamahagung.go.id/direktori/download_file/fe2d384fca8d8f5b7e1f45cb51cea75c/pdf/zaed6bcbde996ccc906f313434343437
https://putusan3.mahkamahagung.go.id/direktori/download_file/5d0e9029be1e162aaa3474ffe8805472/pdf/zaed1d034a759ece9a56303833323238
https://putusan3.mahkamahagung.go.id/direktori/download_file/336966f7fbfe539db5a397

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Download semua file dan disimpan ke Google Drive
for fileurl in all_link:
  filename = os.path.basename(fileurl)
  r = requests.get(fileurl, stream = True)

  with open(f'/content/gdrive/My Drive/Colab Notebooks/Temu Kembali Informasi/Dataset/{filename}.pdf', 'wb') as file:
    for block in r.iter_content(chunk_size = 1024):
      if block:
        file.write(block)